<p style="text-align:center;">
<img src=https://noodle.digitalfutures.com/studentuploads/Data_Cygnets_logo.png width = 150px, height=150px/
     style="float: center; " />
</p>

# Customer Churn Prediction for Swan Teleco
🌲 🌳 Using Random Forest algorithms from Sklearn 🌳🌲
### by Data Cygnets
🦢 Jamie M   
🦢 Muqadas   
🦢 Sennan   
🦢 Maarja

## Imports

In [ ]:
#Numerical information
import numpy as np

#General Data use
import pandas as pd
from collections import Counter

#Data Visualisation
import seaborn as sns
import matplotlib.pyplot as plt


##Predictive Modelling##
#Metrics
from sklearn import metrics
from sklearn.metrics import accuracy_score, precision_score, recall_score, precision_recall_curve, f1_score, roc_curve, auc, classification_report, confusion_matrix
from mlxtend.plotting import plot_confusion_matrix


#Model
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier

#HyperParameters
from sklearn.model_selection import GridSearchCV, StratifiedKFold, train_test_split, cross_val_score

#Class balancing with SMOTE(Synthetic Minority Over-sampling Technique-https://machinelearningmastery.com/smote-oversampling-for-imbalanced-classification/)
#Please run this if the package is not currently installed in the environment:
#!pip install imbalanced-learn -U
import imblearn
from imblearn.pipeline import Pipeline as imbpipeline
from imblearn.over_sampling import SMOTE
#https://medium.com/data-science/the-right-way-of-using-smote-with-cross-validation-92a8d09d00c7#:~:text=We%20first%20split%20the%20data,cross%2Dvalidation%20and%20test%20scores.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

## Load data

In [ ]:
# data
df = pd.read_excel('1 - Project Data.xlsx')

In [ ]:
df.head(2)

## Splitting data

In [ ]:
# splitting into train/test 80:20 split
train, test = train_test_split(df, random_state = 60, stratify = df['Churn Label'], test_size = 0.2)

## Feature Engineering

In [ ]:
# binary columns to be converted to 0/1
Y_N_cols = ['Senior Citizen', 'Partner', 'Dependents', 'Phone Service', 'Multiple Lines', 'Internet Service',
       'Online Security', 'Online Backup', 'Device Protection', 'Tech Support',
       'Streaming TV', 'Streaming Movies', 'Paperless Billing']

categorical_cols = ['City','Contract','Payment Method']

def str_to_int_boolean(x):
    if x=='Yes':
        return 1
    else:
        return 0
def service_count(row):
    return (~row.str.contains('No')).sum()


def feature_eng(df):

    # Removal of columns
    features = list(df.columns)
    # Dropping unnecessary columns
    features.remove('CustomerID')
    features.remove('Count')
    features.remove('Country')
    features.remove('State')
    features.remove('Lat Long')
    features.remove('Latitude')
    features.remove('Longitude')
    # Data leakage
    features.remove('Churn Reason')

    # Duplication
    features.remove('Churn Value')

    df_copy = df[features].copy()
    df_copy.index = df.CustomerID

    ##Cleaning of Data##
    df_copy.replace('No internet service', 'No', inplace=True)
    df_copy.replace('No phone service', 'No', inplace=True)
    df_copy['Total Charges'] = df_copy['Total Charges'].replace(' ', 0)
    df_copy['Total Charges'] = df_copy['Total Charges'].astype(float)
    df_copy = df_copy[df_copy['Total Charges'] != 0]

    # service columns
    cols = ['Phone Service','Multiple Lines', 'Internet Service', 'Online Security',
       'Online Backup', 'Device Protection', 'Tech Support', 'Streaming TV',
       'Streaming Movies']

    # Counting the number of services
    df_copy['service_count'] = df_copy[cols].apply(service_count, axis=1)

    ##Changing from str to integer booleans##
    for i in Y_N_cols:
        df_copy[i] = df_copy[i].apply(str_to_int_boolean)

    df_copy['Gender'] = [1 if val == 'Male' else 0 for val in df_copy['Gender']]

    ##Changing categorical into numerical

    for i in categorical_cols:
        df_copy[i] = pd.factorize(df_copy[i])[0]

    # separating target and features
    y = df_copy['Churn Label'].copy()
    X = df_copy.drop(columns = ['Churn Label']).copy()

    # Keep important features based on feature importance
    top_features = ['Contract', 'Monthly Charges', 'Dependents', 'Tenure Months', 'Payment Method', 'Total Charges',
                    'Tech Support', 'Online Security', 'Zip Code', 'service_count']
    X = X[top_features]

    return X,y

In [ ]:
# Apply FE on the training set
X_train_fe, y_train_fe = feature_eng(train)

In [ ]:
X_train_fe.head() # view changes

In [ ]:
y_train_fe.head()

## Model Development

In [ ]:
# Pipeline applies SMOTE and train a random forest classifier
pipeline = imbpipeline(steps = [['smote', SMOTE(random_state=60)],
                                        ['rf_classifier', RandomForestClassifier(random_state=60)]])

# StratifiedKFold cross-validator with 5 splits
stratified_kfold = StratifiedKFold(n_splits=5,
                                       shuffle=True,
                                       random_state=60)

# hyperparameter grid for tuning
param_grid = {
    'rf_classifier__criterion':['log_loss','entropy','gini'],
    'rf_classifier__n_estimators': [100, 150, 200],
    'rf_classifier__max_depth': [None, 5, 10, 20],
    'rf_classifier__min_samples_split': [2, 5],
    'rf_classifier__min_samples_leaf': [1, 2],
    'rf_classifier__max_features': [None, 'sqrt', 'log2']
}

# set up GridSearchCV to find best params for RF model
grid_search = GridSearchCV(estimator=pipeline,
                            param_grid=param_grid,
                            scoring='roc_auc',
                            cv=stratified_kfold,
                            n_jobs=-1)


grid_search.fit(X_train_fe, y_train_fe)

In [ ]:
grid_search.best_score_

In [ ]:
grid_search.best_params_

In [ ]:
# best model from grid search
top_features_rf = grid_search.best_estimator_

In [ ]:
# predicting on train
y_train_pred = top_features_rf.predict(X_train_fe)

## Model Evaluation

In [ ]:
print("Classification Report on Train:")
cr_train = classification_report(y_train_pred, y_train_fe)
print(cr_train)

In [ ]:
# FE on test
X_test_fe, y_test_fe = feature_eng(test)

In [ ]:
# predicting on test set
y_test_pred = top_features_rf.predict(X_test_fe)

In [ ]:
print("Classification Report on Test:")
cr_test = classification_report(y_test_fe, y_test_pred)
print(cr_test)

**Comparison with previous RF model (Train):**  
✅ 'No' class is still being classified very confidently.  

✅ 'Yes' class has good precision (0.85) → when it predicts churn, it's likely right.  

🛑 Recall is lower (went from 0.68 to 0.66) → it's missing about 34% of churners in training.

**Comparison with previous RF model (Test):**  

✅ 'Yes' recall improved (went from 0.55 (in baseline model) to 0.73 ) → which is great for a churn use case since we're prioritising customers that will churn.  

🆗 Precision for 'Yes' dropped to 0.58 → more false positives, but it's  acceptable in churn if since Swan Teleco's business goal is to proactively target at-risk customers.  

🆗 Overall accuracy is 0.79, which is decent for a recall-heavy model.


**Comparison of Train and Test set results for THIS model:**
- Overfitting is likely: strong performance on training, weaker and less balanced on test.
- The model favours the majority class ('No'), especially in training.
- Test set shows a trade-off: better recall for churners but worse precision (which is to be expected since we're focusing on recall)

## Conclusion
- Random was not selected as the final model as it did not provide the best results despite the tuning and feature selection methods applied.

# Appendix

In [ ]:
# @title ROC Curve

# get predicted probabilities
y_scores = top_features_rf.predict_proba(X_test_fe)[:, 1]

# ROC curve
fpr, tpr, thresholds = roc_curve(y_test_fe, y_scores, pos_label='Yes')
roc_auc = auc(fpr, tpr)

# plot curve
plt.figure(figsize=(8, 6))
plt.plot(fpr, tpr, label=f"ROC Curve (AUC = {roc_auc:.2f})")
plt.plot([0, 1], [0, 1], 'k--', label="Random Guess")
plt.xlabel("False Positive Rate")
plt.ylabel("True Positive Rate")
plt.title("ROC Curve")
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()

In [ ]:
# @title Identify Best Decision Threshold
# predicted probabilities for the positive class Yes
y_test_proba = top_features_rf.predict_proba(X_test_fe)[:, 1]

# precision-recall values for different thresholds
precision, recall, thresholds = precision_recall_curve(y_test_fe, y_test_proba, pos_label='Yes')

# calculate F1 scores for each threshold
f1_scores = 2 * (precision * recall) / (precision + recall + 1e-8)

# find the threshold with the best F1 score
best_idx = f1_scores.argmax()
best_threshold = thresholds[best_idx]

print(f"Best threshold for max F1: {best_threshold:.2f}")
print(f"Precision: {precision[best_idx]:.2f}, Recall: {recall[best_idx]:.2f}, F1: {f1_scores[best_idx]:.2f}")

# precision-recall vs threshold plot
plt.figure(figsize=(10, 6))
plt.plot(thresholds, precision[:-1], label='Precision', color='blue')
plt.plot(thresholds, recall[:-1], label='Recall', color='green')
plt.plot(thresholds, f1_scores[:-1], label='F1 Score', color='red')
plt.axvline(x=best_threshold, color='black', linestyle='--', label=f'Best Threshold ({best_threshold:.2f})')
plt.xlabel('Threshold')
plt.ylabel('Score')
plt.title('Precision, Recall & F1 vs Threshold')
plt.legend()
plt.grid(True)
plt.show()

### Feature Importance from previous model

In [ ]:
# # get the model inside the pipeline
# rf_model = best_rf.named_steps['rf_classifier']

# # get feature importances
# importances = rf_model.feature_importances_
# features = X_train_fe.columns
# indices = np.argsort(importances)[::-1]

# # features sorted by importance
# for i in indices:
#     print(f"{features[i]}: {importances[i]:.4f}")